time in bond and stock are upside down

gom laij me thanh 1 table

FINAL PROJECT: BLEBLBLEBEL

Group bleble

Team members:

In [3]:
# Import necessary libraries:

import pandas as pd


Create a table including all data points

In [ ]:
# File paths for datasets
# gold = pd.read_csv("data/gold_prices_monthly.csv")      # gold_price  
bond_file_path = "data/bond_yield_daily.csv"        
stock_file_path = pd.read_csv("data/stock_index_daily.csv")      

# Make sure date columns are parsed correctly

# stock["date"] = pd.to_datetime(stock["Date"])

df = pd.read_csv(bond_file_path, parse_dates=['Ngày'], dayfirst=True)

# Ensure the "Lần cuối" column is numeric
df['Lần cuối'] = pd.to_numeric(df['Lần cuối'], errors='coerce')

# Group by month and year, compute the average
monthly_avg = df.groupby(df['Ngày'].dt.to_period('M'))['Lần cuối'].mean().reset_index()

# Format the 'date' column as MM/YYYY
monthly_avg['date'] = monthly_avg['Ngày'].dt.strftime('%m/%Y')

# Rename columns
monthly_avg = monthly_avg[['date', 'Lần cuối']].rename(columns={'Lần cuối': 'bond_yield'})

print(monthly_avg)


# -----------------------------
# 2. Reverse bond & stock to match gold format
# -----------------------------
# bond = bond.sort_values("date", ascending=True).reset_index(drop=True)
# stock = stock.sort_values("date", ascending=True).reset_index(drop=True)

# -----------------------------
# 3. Merge everything into one table
# -----------------------------
# df = gold.merge(
#         bond[["date", "Price"]].rename(columns={"Price": "bond_yield"}),
#         on="date",
#         how="left"
#      ).merge(
#         stock[["date", "Price"]].rename(columns={"Price": "stock_index"}),
#         on="date",
#         how="left"
#      )

# -----------------------------
# 4. Preview the merged dataset
# -----------------------------
# print(df.head())

        date  bond_yield
0    10/2015    7.034208
1    11/2015    7.073727
2    12/2015    7.059652
3    01/2016    7.056091
4    02/2016    7.007045
..       ...         ...
116  06/2025    3.281429
117  07/2025    3.381652
118  08/2025    3.564571
119  09/2025    3.702182
120  10/2025    3.761000

[121 rows x 2 columns]


In [14]:
import pandas as pd

# --- Step 1: Load the bond yield data ---
bond_file = "data/bond_yield_daily.csv"
bond_df = pd.read_csv(bond_file, parse_dates=['Ngày'], dayfirst=True)
bond_df['Lần cuối'] = pd.to_numeric(bond_df['Lần cuối'], errors='coerce')

# Compute monthly average for bond yield
bond_monthly_avg = bond_df.groupby(bond_df['Ngày'].dt.to_period('M'))['Lần cuối'].mean().reset_index()
bond_monthly_avg['date'] = bond_monthly_avg['Ngày'].dt.strftime('%m/%Y')
bond_monthly_avg = bond_monthly_avg[['date', 'Lần cuối']].rename(columns={'Lần cuối': 'bond_yield'})

# --- Step 2: Load the stock index data ---
stock_file = "data/stock_index_daily.csv"
stock_df = pd.read_csv(stock_file, parse_dates=['Ngày'], dayfirst=True)
# Remove commas and convert to float
stock_df['Lần cuối'] = stock_df['Lần cuối'].str.replace(',', '', regex=False)
stock_df['Lần cuối'] = pd.to_numeric(stock_df['Lần cuối'], errors='coerce')


# Compute monthly average for stock index
stock_monthly_avg = stock_df.groupby(stock_df['Ngày'].dt.to_period('M'))['Lần cuối'].mean().reset_index()
stock_monthly_avg['date'] = stock_monthly_avg['Ngày'].dt.strftime('%m/%Y')
stock_monthly_avg = stock_monthly_avg[['date', 'Lần cuối']].rename(columns={'Lần cuối': 'stock_index'})

# --- Step 3: Merge bond and stock monthly averages ---
monthly_avg['month_period'] = pd.to_datetime(monthly_avg['date'], format='%m/%Y')
monthly_avg = monthly_avg.sort_values('month_period').drop(columns='month_period').reset_index(drop=True)

monthly_avg = pd.merge(bond_monthly_avg, stock_monthly_avg, on='date', how='outer')

print(monthly_avg)


        date  bond_yield  stock_index
0    01/2016    7.056091   548.304500
1    01/2017    6.053318   683.260588
2    01/2018    4.663913  1058.376190
3    01/2019    5.000391   902.198636
4    01/2020    3.098591   966.498824
..       ...         ...          ...
116  12/2020    2.351957  1057.952174
117  12/2021    2.087609  1472.195652
118  12/2022    5.121909  1035.679091
119  12/2023    2.420095  1114.568095
120  12/2024    3.002409  1264.157273

[121 rows x 3 columns]
